<a href="https://colab.research.google.com/github/yurizza/H8DSFT_NGC/blob/main/h8dsft_P0W3_cici_yuriza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graded Challange 3

- Name : Cici Yuriza
- Batch : HCK-005


## Description

Objective :\
Meningkatkan total pendapatan 50% Q4 2022 pada e-commerce thelook.


Problem Statement :\
Thelook e-commerce menganalisa pendapatan dengan category dan Status 'Complete' pada Periode Q1 hingga Q3 2022 sehingga tim stok barang akan memperbanyak stok dan mengurangi stok item tertentu untuk meningkatkan pendapatan 50% pada Q4 2022.

### Connect to Google Cloud Platform

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
project_id = "ciciyuriza" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

### Data Loading

In [3]:
# import library
import pandas as pd

In [4]:
# menampilkan semua data dalam tabel orders
df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
''').to_dataframe()

In [5]:
# menampilkan 5 data teratas
df.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,12,12,Shipped,F,2022-07-01 02:23:00+00:00,NaT,2022-07-03 02:02:00+00:00,NaT,2
1,29,27,Shipped,F,2022-02-11 06:13:00+00:00,NaT,2022-02-11 14:05:00+00:00,NaT,1
2,34,32,Shipped,F,2022-09-21 05:44:00+00:00,NaT,2022-09-23 06:22:00+00:00,NaT,1
3,43,41,Shipped,F,2022-02-08 11:26:00+00:00,NaT,2022-02-10 11:52:00+00:00,NaT,1
4,46,45,Shipped,F,2019-04-20 09:46:00+00:00,NaT,2019-04-22 08:00:00+00:00,NaT,1


In [6]:
# menampilkan 5 data terakhir
df.tail()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
124519,124461,99934,Processing,M,2022-11-07 14:12:00+00:00,NaT,NaT,NaT,2
124520,124471,99946,Processing,M,2022-06-10 05:23:00+00:00,NaT,NaT,NaT,1
124521,124508,99984,Processing,M,2019-04-03 12:56:00+00:00,NaT,NaT,NaT,3
124522,124509,99984,Processing,M,2021-06-26 12:56:00+00:00,NaT,NaT,NaT,3
124523,124521,99996,Processing,M,2020-04-17 14:36:00+00:00,NaT,NaT,NaT,1


Informasi dari tabel :
terdapat 124524 transaksi dalam tabel orders.

Hasil analisis tabel-tabel dalam dataset thelook_ecommerce :

1. Tabel order_items merupakan penjabaran dari tabel orders.
Misal user melakukan pembelian 2 barang maka akan dicatat pada tabel orders. Untuk detail harga dari kedua barang maka ada pada tabel order_items.
2. Tabel orders, mencatat transaksi yang terjadi dan jumlah barang yang di pesan.
3. Tabel products, menampilkan informasi produk termasuk category produk.

### Poin penjabaran

1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [7]:
# menampilkan jumlah order per bulan yang berstatus 'Complete'
# rentang januari - september 2022
client.query('''
SELECT EXTRACT(month FROM created_at) As bulan, COUNT(order_id) as Jumlah_order_per_month_2022_status_complete
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE Status LIKE 'Complete' AND (created_at BETWEEN '2022-01-01' AND '2022-10-01')
GROUP BY bulan
ORDER BY bulan
''').to_dataframe()

,bulan,Jumlah_order_per_month_2022_status_complete
0,1,731
1,2,639
2,3,798
3,4,814
4,5,896
5,6,949
6,7,1058
7,8,1143
8,9,1139


Jumlah order terus mengalami peningkatan dari bulan Januari hingga September 2022.

2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [8]:
# menampilkan total pendapatan (USD) perbulan
# rentang januari - september 2022
client.query('''
SELECT EXTRACT(month FROM created_at) AS bulan, SUM(sale_price) AS total_penjualan_USD
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE Status LIKE 'Complete' AND (created_at BETWEEN '2022-01-01' AND '2022-10-01')
GROUP BY bulan
ORDER BY bulan
''').to_dataframe()

,bulan,total_penjualan_USD
0,1,66960.140152
1,2,53642.600047
2,3,69355.180078
3,4,73085.630079
4,5,69848.740080
5,6,82351.620036
6,7,93887.750075
7,8,94100.440072
8,9,100529.240070


Total pendapatan terbanyak dari Januari hingga September 2022 adalah pada bulan Agustus.

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [9]:
# menampilkan jumlah user yang melakukan transaksi berstatus 'Complete' setiap bulan
# rentang januari - september 2022.
client.query('''
SELECT EXTRACT(month FROM created_at) AS bulan, COUNT(DISTINCT(user_id)) AS jumlah_user_status_complete
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE Status LIKE 'Complete' AND (created_at BETWEEN '2022-01-01' AND '2022-10-01')
GROUP BY bulan
ORDER BY bulan
''').to_dataframe()

,bulan,jumlah_user_status_complete
0,1,728
1,2,638
2,3,793
3,4,812
4,5,887
5,6,944
6,7,1055
7,8,1134
8,9,1128


Jumlah user yang melakukan transaksi terus bertambah setiap bulan dari Januari hingga September 2022. Jika dibandingkan dengan jumlah order per month (soal no 1) maka ada user yang melakukan pembelian lebih dari satu kali setiap bulannya.

4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [10]:
# set maximum display column 
pd.set_option('display.max_columns', None)

In [11]:
# membuat pivot tabel bulan terhadap jumlah item yang terjual perkategori dengan status 'Complete' 
# rentang januari - september 2022.
client.query('''
SELECT bulan,
       SUM(jumlah) AS jumlah_produk,
       SUM(CASE WHEN category = 'Accessories' THEN jumlah ELSE NULL END) AS Accessories,
       SUM(CASE WHEN category = 'Active' THEN jumlah ELSE NULL END) AS Active,
       SUM(CASE WHEN category = 'Blazers & Jackets' THEN jumlah ELSE NULL END) AS Blazers_and_Jackets,
       SUM(CASE WHEN category = 'Clothing Sets' THEN jumlah ELSE NULL END) AS Clothing_Sets,
       SUM(CASE WHEN category = 'Dresses' THEN jumlah ELSE NULL END) AS Dresses,
       SUM(CASE WHEN category = 'Fashion Hoodies & Sweatshirts' THEN jumlah ELSE NULL END) AS Fashion_Hoodies_and_Sweatshirts,
       SUM(CASE WHEN category = 'Intimates' THEN jumlah ELSE NULL END) AS Intimates,
       SUM(CASE WHEN category = 'Jeans' THEN jumlah ELSE NULL END) AS Jeans,
       SUM(CASE WHEN category = 'Jumpsuits & Rompers' THEN jumlah ELSE NULL END) AS Jumpsuits_and_Rompers,
       SUM(CASE WHEN category = 'Leggings' THEN jumlah ELSE NULL END) AS Leggings,
       SUM(CASE WHEN category = 'Maternity' THEN jumlah ELSE NULL END) AS Maternity,
       SUM(CASE WHEN category = 'Outerwear & Coats' THEN jumlah ELSE NULL END) AS Outerwear_and_Coats,
       SUM(CASE WHEN category = 'Pants' THEN jumlah ELSE NULL END) AS Pants,
       SUM(CASE WHEN category = 'Pants & Capris' THEN jumlah ELSE NULL END) AS Pants_and_Capris,
       SUM(CASE WHEN category = 'Plus' THEN jumlah ELSE NULL END) AS Plus,
       SUM(CASE WHEN category = 'Shorts' THEN jumlah ELSE NULL END) AS Shorts,
       SUM(CASE WHEN category = 'Skirts' THEN jumlah ELSE NULL END) AS Skirts,
       SUM(CASE WHEN category = 'Sleep & Lounge' THEN jumlah ELSE NULL END) AS Sleep_and_Lounge,
       SUM(CASE WHEN category = 'Socks' THEN jumlah ELSE NULL END) AS Socks,
       SUM(CASE WHEN category = 'Socks & Hosiery' THEN jumlah ELSE NULL END) AS Socks_and_Hosiery,
       SUM(CASE WHEN category = 'Suits' THEN jumlah ELSE NULL END) AS Suits,
       SUM(CASE WHEN category = 'Suits & Sport Coats' THEN jumlah ELSE NULL END) AS Suits_and_Sport_Coats,
       SUM(CASE WHEN category = 'Sweaters' THEN jumlah ELSE NULL END) AS Sweaters,
       SUM(CASE WHEN category = 'Swim' THEN jumlah ELSE NULL END) AS Swim,
       SUM(CASE WHEN category = 'Tops & Tees' THEN jumlah ELSE NULL END) AS Tops_and_Tees,
       SUM(CASE WHEN category = 'Underwear' THEN jumlah ELSE NULL END) AS Underwear
  FROM (
        SELECT EXTRACT(month FROM items.created_at) AS bulan, products.category AS category, COUNT(items.id) AS jumlah
        FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
        On items.product_id = products.id
        WHERE Status LIKE 'Complete' AND (items.created_at BETWEEN '2022-01-01' AND '2022-10-01')
        GROUP BY bulan, products.category
        ORDER BY bulan
       ) sub
 GROUP BY 1
 ORDER BY 1 
 ''').to_dataframe()

,bulan,jumlah_produk,Accessories,Active,Blazers_and_Jackets,Clothing_Sets,Dresses,Fashion_Hoodies_and_Sweatshirts,Intimates,Jeans,Jumpsuits_and_Rompers,Leggings,Maternity,Outerwear_and_Coats,Pants,Pants_and_Capris,Plus,Shorts,Skirts,Sleep_and_Lounge,Socks,Socks_and_Hosiery,Suits,Suits_and_Sport_Coats,Sweaters,Swim,Tops_and_Tees,Underwear
0,1,1093,53,52,14,<NA>,33,62,88,78,7,20,40,47,50,20,14,69,13,72,42,23,5,27,74,74,74,42
1,2,922,50,55,14,1,26,57,83,62,4,18,24,54,34,20,30,59,8,49,28,21,5,17,44,65,58,36
2,3,1195,69,52,22,1,24,73,107,76,8,15,24,71,34,16,33,69,14,81,49,25,8,23,76,70,86,69
3,4,1183,64,60,22,1,42,74,84,76,4,18,42,67,43,22,24,73,12,70,44,22,8,43,72,93,55,48
4,5,1263,69,66,18,<NA>,31,84,109,73,6,26,42,56,63,35,24,74,11,73,40,20,8,33,82,65,108,47
5,6,1374,71,69,23,2,37,84,108,88,8,24,32,66,58,21,37,92,24,84,44,28,4,50,92,77,93,58
6,7,1522,78,86,25,3,56,93,112,105,12,30,58,85,53,23,35,95,15,92,43,25,8,47,96,93,89,65
7,8,1581,107,79,29,3,45,87,109,99,9,22,45,92,41,29,43,112,18,106,50,32,8,37,84,119,103,73
8,9,1646,82,87,27,3,42,112,129,122,6,28,46,83,68,25,34,108,20,102,49,34,19,59,120,80,94,67


In [12]:
# mengambil category dan jumlah item per bulan 
# kemudian mengambil nilai max dari jumlah item setiap bulan
# menampilkan category-category yang jumlah item = max jumlah item setiap bulan
client.query('''
SELECT details.bulan,details.category, details.item
FROM(  
  SELECT bulan, MAX(item) AS maxim
  FROM(
    SELECT EXTRACT(month FROM items.created_at) AS bulan, 
    products.category category, 
    COUNT(items.id) as item,
    FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
    JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
    On items.product_id = products.id
    WHERE Status LIKE 'Complete' AND (items.created_at BETWEEN '2022-01-01' AND '2022-10-01')
    GROUP BY bulan, products.category
    ORDER BY bulan ASC, item DESC
    )
  GROUP BY bulan
  ORDER BY bulan
) AS maxi
JOIN(
  SELECT bulan, category, item
  FROM(
    SELECT EXTRACT(month FROM items.created_at) AS bulan, 
    products.category category, 
    COUNT(items.id) as item,
    FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
    JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
    On items.product_id = products.id
    WHERE Status LIKE 'Complete' AND (items.created_at BETWEEN '2022-01-01' AND '2022-10-01')
    GROUP BY bulan, products.category
    ORDER BY bulan ASC, item DESC
  )
) AS details
ON details.bulan = maxi.bulan
WHERE maxi.maxim = details.item
ORDER BY details.bulan
''').to_dataframe()

,bulan,category,item
0,1,Intimates,88
1,2,Intimates,83
2,3,Intimates,107
3,4,Swim,93
4,5,Intimates,109
5,6,Intimates,108
6,7,Intimates,112
7,8,Swim,119
8,9,Intimates,129


Category Intimates menjadi produk paling banyak dibeli pada 7 bulan pada 2022. 2 bulan (bulan ke 4 dan 8) category swim.

5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [13]:
# mengambil category, jumlah item per category dan jumlah pendapatan per bulan 
# kemudian mengambil nilai max dari pendapatan setiap bulan
# menampilkan jumlah item dan category-category yang Pendapatan = max pendapatan setiap bulan
client.query('''
SELECT details.bulan,details.category, details.item, details.pendapatan
FROM(  
  SELECT bulan, MAX(pendapatan) AS max_pendapatan
  FROM(
    SELECT EXTRACT(month FROM items.created_at) AS bulan, 
    SUM(items.sale_price) AS pendapatan,
    products.category category, 
    COUNT(items.id) as item,
    FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
    JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
    On items.product_id = products.id
    WHERE Status LIKE 'Complete' AND (items.created_at BETWEEN '2022-01-01' AND '2022-10-01')
    GROUP BY bulan, products.category
    ORDER BY bulan ASC, item DESC
    )
  GROUP BY bulan
  ORDER BY bulan
) AS maxi
JOIN(
  SELECT bulan, category, item, pendapatan
  FROM(
    SELECT EXTRACT(month FROM items.created_at) AS bulan, 
    SUM(items.sale_price) AS pendapatan,
    products.category category, 
    COUNT(items.id) as item,
    FROM `bigquery-public-data.thelook_ecommerce.order_items` AS items
    JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
    On items.product_id = products.id
    WHERE Status LIKE 'Complete' AND (items.created_at BETWEEN '2022-01-01' AND '2022-10-01')
    GROUP BY bulan, products.category
    ORDER BY bulan ASC, pendapatan DESC
  )
) AS details
ON details.bulan = maxi.bulan
WHERE maxi.max_pendapatan = details.pendapatan
ORDER BY bulan
  ''').to_dataframe()

,bulan,category,item,pendapatan
0,1,Jeans,78,8166.550024
1,2,Outerwear & Coats,54,8956.509996
2,3,Outerwear & Coats,71,9049.000034
3,4,Outerwear & Coats,67,11264.800016
4,5,Outerwear & Coats,56,8177.499994
5,6,Outerwear & Coats,66,10299.570013
6,7,Outerwear & Coats,85,12599.140013
7,8,Outerwear & Coats,92,11872.089951
8,9,Jeans,122,11309.390007


Pendapatan paling banyak selama 7 bulan yaitu outwear & coats. dan Jeans selama 2 bulan. Pendapatan paling tinggi pada bulan 7 dengan category outwear & coats $12599.140013.

6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

In [14]:
# query jumlah transaksi, total penjualan dan jumlah user setiap bulan dari januari - september 2022 
df = client.query('''
SELECT EXTRACT(month FROM created_at) AS bulan, 
        COUNT(DISTINCT(order_id)) AS jumlah_transaksi, 
        COUNT(DISTINCT(user_id)) AS jumlah_user, 
        SUM(sale_price) AS pendapatan
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE Status LIKE 'Complete' AND (created_at BETWEEN '2022-01-01' AND '2022-10-01')
GROUP BY bulan
ORDER BY bulan
''').to_dataframe()

In [15]:
# menampilkan data secara berurut
df.sort_values(by=['jumlah_transaksi','jumlah_user','pendapatan'], ascending=False)

,bulan,jumlah_transaksi,jumlah_user,pendapatan
8,9,1163,1150,100529.240070
7,8,1155,1146,94100.440072
6,7,1073,1070,93887.750075
5,6,967,962,82351.620036
4,5,901,892,69848.740080
3,4,835,832,73085.630079
2,3,814,810,69355.180078
0,1,740,737,66960.140152
1,2,653,652,53642.600047


In [16]:
# melihat korelasi antar kolom
df.corr()

,bulan,jumlah_transaksi,jumlah_user,pendapatan
bulan,1.000000,0.969875,0.969134,0.925487
jumlah_transaksi,0.969875,1.000000,0.999891,0.974827
jumlah_user,0.969134,0.999891,1.000000,0.976188
pendapatan,0.925487,0.974827,0.976188,1.000000


Korelasi antara user dengan jumlah transaksi 0.9999891.\
Korelasi antara user dengan pendapatan 0.976188.

Sehingga dari bulan januari hingga september 2022, orders yang status complete korelasi antara jumlah user dengan jumlah transaksi lebih besar dari pada user dengan pendapatan. 

`Makin banyak user melakukan order maka makin banyak transaksi dan makin banyak pendapatan.`

### Poin Analisis

Berdsarkan hasil analisis maka tim penjualan akan melakukan penaikan pendapatan sebesar 50% pada Q4 dengan memperbanyak penjualan `jeans & outwear_and_coats` karena keduanya menaikan pendapatan.Selain itu tim juga akan memperbanyak item dengan kategori `intimates dan swim` karena permintaan dari user yang tinggi pada kedua kategori ini.

### Poin Pertanyaan

1. Iya, mari kita coba hitung :
> Q1 = 189957\
Q2 = 225284\
Q3 = 288516\
Selisih Q2 dengan Q1 = 35327 (S1)\
selisih Q3 dengan Q2 = 63232 (S2)\
Selisih S2 dengan S1 = 27905 (S3)\
Maka S2 + S3 = 91137 (kenaikan yang akan terjadi pada Q4) (x)\
x / Q3 = persentase kenaikan Q4\
91137/288516 ~ 0.32 ~ 32%
Maka hanya butuh menaikan 20% saja untuk mencapai target.
2. Pendapatan setiap bulan meningkat namun tidak sampai 2x lipat dari bulan sebelumnya. Pendapatan pada bulan September mencapai 98954.520119. Sehingga dibutuhkan banyak sekali evaluasi untuk meningkatkan pendapatan lebih dari 100% diawal Q4. Oleh karena itu **tidak masuk akal**.
3. GMV = jumlah barang terjual * harga barang
> Q1 = 189957\
Q2 = 225284\
Q3 = 288516\
GMV = Q1+Q2+Q3 = 703757\
Jumlah user = 7767 (menghitung jumlah user yang melakukan transaksi selama Q1-Q3 2022)\
trend transaksi = item dengan kategori intimates dan swim.\
Minimal pendapatan pada Q4 adalah 379653 atau 30% lebih tinggi dari Q3.

In [17]:
# menghitung jumlah user yang bertransaksi Q1-Q3 2022
client.query('''
SELECT count(DISTINCT(user_id)) AS jumlah_user
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE Status LIKE 'Complete' AND (created_at BETWEEN '2022-01-01' AND '2022-10-01')
''').to_dataframe()

,jumlah_user
0,7767


### OVERALL CONCLUSION

Pada Q4 akan terjadi peningkatan pendapatan minimal 32% tanpa harus melakukan perubahan pada item yang dijual sebelumnya. Apabila tim penjualan melakukan perubahan dengan meningkatkan barang dengan kategori tertentu (Jeans, outwear & coats, intimates dan swim) pasti akan mencapai 50% bahkan bisa lebih. Tambahan investor mungkin akan meningkatkan pendapatan.